In [1]:
# pip install pandas geopandas matplotlib contextily

import pandas as pd
import geopandas as gpd
from datetime import timedelta
import contextily as ctx
import matplotlib.pyplot as plt


In [2]:

# Charger les données
file_path = "dataset_tsmc2014/dataset_TSMC2014_NYC.csv"  # Remplacez par le chemin vers votre fichier
data = pd.read_csv(file_path)

# Convertir les timestamps en format datetime
data['utcTimestamp'] = pd.to_datetime(data['utcTimestamp'], format='%a %b %d %H:%M:%S %z %Y')

# Trier les données par lieu et timestamp
data_sorted = data.sort_values(by=['venueId', 'utcTimestamp']).reset_index(drop=True)

# Créer une copie des données pour effectuer une jointure sur les lieux
data_copy = data_sorted.copy()

# Jointure pour trouver les utilisateurs ayant visité le même lieu
merged = data_sorted.merge(
    data_copy, 
    on="venueId", 
    suffixes=("_1", "_2")
)

# Calculer la différence de temps
merged['time_diff'] = abs(merged['utcTimestamp_1'] - merged['utcTimestamp_2'])

# Filtrer les interactions à moins d'une heure entre des utilisateurs différents
filtered = merged[
    (merged['userId_1'] != merged['userId_2']) & (merged['time_diff'] <= timedelta(hours=1))
]

# Normaliser les paires (toujours avoir userId_1 < userId_2)
filtered['user1'] = filtered[['userId_1', 'userId_2']].min(axis=1)
filtered['user2'] = filtered[['userId_1', 'userId_2']].max(axis=1)

# Compter les interactions par paire unique
pair_counts = (
    filtered.groupby(['user1', 'user2'])
    .size()
    .reset_index(name='interaction_count')
)

# Filtrer les paires avec au moins 10 interactions
frequent_pairs = pair_counts[pair_counts['interaction_count'] >= 10].reset_index(drop=True)

# Afficher les utilisateurs qui peuvent se connaître
print(frequent_pairs)


# Extraire les lieux associés aux interactions
top_pairs = frequent_pairs.head(5)

# Extraire les lieux associés aux interactions des paires sélectionnées
top_venues = data[data['venueId'].isin(filtered['venueId'])][['venueId', 'latitude', 'longitude']].drop_duplicates()

# Convertir en GeoDataFrame
gdf = gpd.GeoDataFrame(
    top_venues, 
    geometry=gpd.points_from_xy(top_venues.longitude, top_venues.latitude),
    crs="EPSG:4326"
)

# Reprojection en Web Mercator
gdf = gdf.to_crs(epsg=3857)

# Tracer les lieux sur une carte
fig, ax = plt.subplots(figsize=(12, 8))
gdf.plot(ax=ax, color="blue", markersize=50, label="Lieu")
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)

# Annoter les interactions
for i, row in top_pairs.iterrows():
    user1, user2 = row['user1'], row['user2']
    venues = filtered[
        (filtered['user1'] == user1) & (filtered['user2'] == user2)
    ]['venueId'].unique()
    for venue in venues:
        location = gdf[gdf['venueId'] == venue].geometry.iloc[0]
        ax.annotate(f"{user1}-{user2}", (location.x, location.y), fontsize=8, color="red")

plt.title("Interactions entre utilisateurs (5 premières paires)")
plt.legend()
plt.show()